In [1]:
import dask
df = dask.datasets.timeseries()

In [2]:
df

,name,id,x,y
npartitions=30,,,,
2000-01-01,object,int64,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=2, memory_limit="1GB")
client

2023-01-04 22:30:20,293 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ixenab1x', purging
2023-01-04 22:30:20,293 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ik56zkej', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43503,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:37801,Total threads: 2
Dashboard: http://127.0.0.1:45257/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:38493,


In [4]:
df2 = df[df.y > 0]
df3 = df2.groupby("name").x.std()
df3


Dask Series Structure:
npartitions=1
    float64
        ...
Name: x, dtype: float64
Dask Name: sqrt, 9 graph layers

In [5]:
computed_df = df3.compute()
type(computed_df)

pandas.core.series.Series

In [6]:
computed_df

name
Alice       0.575638
Bob         0.575876
Charlie     0.576512
Dan         0.575231
Edith       0.576530
Frank       0.578185
George      0.578469
Hannah      0.579137
Ingrid      0.578937
Jerry       0.578421
Kevin       0.577456
Laura       0.576002
Michael     0.578459
Norbert     0.576723
Oliver      0.575684
Patricia    0.577860
Quinn       0.577788
Ray         0.577912
Sarah       0.577027
Tim         0.577864
Ursula      0.577965
Victor      0.578942
Wendy       0.578646
Xavier      0.577135
Yvonne      0.577242
Zelda       0.578219
Name: x, dtype: float64

In [7]:
from sklearn.linear_model import LinearRegression


def train(partition):
    if not len(partition):
        return
    est = LinearRegression()
    est.fit(partition[["x"]].values, partition.y.values)
    return est

